<a href="https://colab.research.google.com/github/Elwing-Chou/ml0716/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [6]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/0724test/chinese_news_trans.zip")
f.extractall("train")
f = zipfile.ZipFile("drive/My Drive/0724test/chinese_news_test.zip")
f.extractall("test")

In [11]:
import glob
import os
import pandas as pd

def getdf(targetdn):
    datas = {
        "path":[],
        "label":[]
    }
    base = ""
    p = os.path.join(base, targetdn, "*")
    for dn in glob.glob(p):
        name = os.path.split(dn)[-1]
        fn = os.path.join(dn, "*.[tT][xX][tT]")
        fns = glob.glob(fn)
        datas["label"] = datas["label"] + [name] * len(fns)
        datas["path"] = datas["path"] + fns
    return pd.DataFrame(datas)

In [12]:
train_df = getdf("train/chinese_news_trans")
test_df = getdf("test/chinese_news_test")

In [14]:
def read_content(fn):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)

In [16]:
test_df

,path,label,content
0,test/chinese_news_test/軍事/8240.txt,軍事,美海軍成功演示飛機在空中控制無人機(附圖) \n\n--------------------...
1,test/chinese_news_test/軍事/8249.txt,軍事,\n軍事評論：“巴格達之戰”考驗美軍巷戰武器裝備(2) \n\n--------------...
2,test/chinese_news_test/軍事/8245.txt,軍事,軍事評論：美英聯軍地面作戰裝備特點分析(2) \n\n-------------------...
3,test/chinese_news_test/軍事/8244.txt,軍事,在伊拉克戰爭中使用的空襲精確制導彈藥(1) \n\n--------------------...
4,test/chinese_news_test/軍事/8247.txt,軍事,軍事評論：從巷戰或地道戰看單兵夜視裝備的發展 \n\n-------------------...
...,...,...,...
96,test/chinese_news_test/政治/9505.TXT,政治,楊主席提出促進中拉關係的５項建議\n新華社蒙得維的亞５月２３日電正在這...
97,test/chinese_news_test/政治/9501.TXT,政治,阿拉伯外長會議閉幕\n新華社巴格達５月２４日電為期２天的阿拉伯外長會\...
98,test/chinese_news_test/政治/9500.TXT,政治,外事簡訊\n新華社北京５月２４日電外事簡訊\n 全國人大常委會副...
99,test/chinese_news_test/政治/9497.TXT,政治,第３次安第斯集團首腦會議閉幕\n新華社庫斯科（祕魯）５月２３日電第３次...


In [ ]:
u = train_df["label"].unique()
trans = {n:i for i, n in enumerate(u)}
trans_r = {i:n for i, n in enumerate(u)}
y_train = train_df["label"].replace(trans)
y_test = test_df["label"].replace(trans)
y_test

In [ ]:
import jieba
from urllib.request import urlretrieve
url = "https://github.com/fxsjy/jieba/raw/master/extra_dict/dict.txt.big"
urlretrieve(url, "dict.txt.big")
jieba.set_dictionary("dict.txt.big")
def newscut(p):
    return " ".join(jieba.cut(p))
x_train = train_df["content"].apply(newscut)
x_test = test_df["content"].apply(newscut)
x_test

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
vec = CountVectorizer()
x_train_count = vec.fit_transform(x_train)
x_test_count = vec.transform(x_test)

In [20]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit(x_train_count, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [21]:
from sklearn.metrics import accuracy_score
pre = clf.predict(x_test_count)
accuracy_score(pre, y_test)

1.0

In [25]:
test = input("輸入一首新聞:")
count = vec.transform([newscut(test)])
answer = clf.predict(count)[0]
proba = clf.predict_proba(count)[0]
for name, p in zip(u, proba):
    print(name, "的機率:", p)
print("答案:", trans_r[answer])

輸入一首新聞:威力彩連摃頭獎直飆25億元，成為全台熱門話題，日前網傳「電腦選號」是送錢給彩券行，對此，台灣彩券推翻民間說法表示，第四屆彩券自103年1月1日起，威力彩頭獎中獎者「電腦選號」比例是70%，近6年來幸運中頭獎的電選與自選比例約為7：3。  PTT近期上有一篇「威力彩電腦選號真的有差嗎？」引起話題，發文者表示，自己路過彩券行順手買了一張威力彩，父親知道他是用電腦選號後就碎念「電腦選號是給彩券行送錢，自己選號才有機會中」，該名父親質疑，電腦選號是真的隨機嗎？也有可能懷疑是設計好1萬組號碼隨機發，且這1萬組號碼永遠都不會中大獎，因此認為電腦選號就是送錢給彩券行。  網友們議論，有人認同「好像有點道理」，但也有人無法相信此民間說法，還有人點出關鍵「開獎也不是真隨機啊，入球順序也會影響結果」，也有人建議乾脆自己用程式跑隨機自己「電腦選號」。  根據「三立新聞網」報導，台彩依據投注系統的統計資料表示，威力彩第四屆的投注交易中，「電腦選號」投注的消費者佔比69.9%，而開出的30注頭獎裡，有21注是「電腦選號」，佔比70%。  另外，消費者購買大樂透和威力彩採「電腦選號」（包括部分電腦選號）與「自行選號」的比例約是7：3，近6年來中頭獎的「電腦選號」與「自行選號」比例，同樣約是7：3。
軍事 的機率: 1.604838448365467e-28
交通 的機率: 2.5785660104385224e-57
計算機 的機率: 1.0
環境 的機率: 1.6901931661840217e-54
藝術 的機率: 4.1248236788596004e-38
醫藥 的機率: 3.1137848484112106e-65
教育 的機率: 6.12773529558314e-39
經濟 的機率: 1.68358224916136e-21
體育 的機率: 3.4214920064480707e-54
政治 的機率: 4.88233768062496e-75
答案: 計算機
